# Practice 3. 빅데이터 분석 실습 (작업형 제2유형)
---

## 📝 예제 문제 : Penguins

- 2014년 남극 펭귄 공동체 내 생태학적 성적 이형성과 환경적 변동성에 관한 연구 데이터
- 다양한 종의 펭귄들의 서식지와 주요 신체 크기와 무게에 대한 7가지 정도의 데이터로 이루어져 있다.
- `Palmer's Penguin`이라는 데이터 세트로도 알려져 있으며, 시각화 패키지 중 하나인 `seaborn` 패키지에 예제 데이터 세트로 존재한다.
- 이를 기반으로 펭귄의 무게를 예측하는 **회귀 분석**을 진행해보자.

### 제공 데이터 목록

- `penguin_y_train.csv` : 펭귄의 무게 데이터(학습용), csv 형식
- `penguin_X_train.csv`, `penguin_X_test.csv` : 펭귄의 속성(학습용 및 평가용), csv 형식

In [2]:
X_train

NameError: name 'X_train' is not defined

In [ ]:
y_train

,body_mass_g
0,4250.0
1,4650.0
2,4250.0
3,4500.0
4,5700.0
...,...
235,3800.0
236,5950.0
237,3200.0
238,3575.0


In [ ]:
X_test

,species,island,sex,bill_length_mm,bill_depth_mm,flipper_length_mm
0,Adelie,Torgersen,MALE,42.1,19.1,195.0
1,Gentoo,Biscoe,MALE,45.5,15.0,220.0
2,Adelie,Biscoe,MALE,40.6,18.8,193.0
3,Adelie,Dream,FEMALE,39.5,17.8,188.0
4,Gentoo,Biscoe,FEMALE,45.1,14.5,207.0
...,...,...,...,...,...,...
96,Gentoo,Biscoe,MALE,45.2,15.8,215.0
97,Adelie,Torgersen,FEMALE,40.3,18.0,195.0
98,Gentoo,Biscoe,FEMALE,46.5,14.5,213.0
99,Gentoo,Biscoe,MALE,49.6,15.0,216.0


## 👨🏻‍💻 문제 풀기

### 패키지, 데이터 불러오기

In [3]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')    # 코드 실행에 직접적인 문제가 되지 않는 경고 문구 무시하기

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
X_test = pd.read_csv('./datasets/Part2/penguin_X_test.csv')
X_train = pd.read_csv('./datasets/Part2/penguin_X_train.csv')
y_train = pd.read_csv('./datasets/Part2/penguin_y_train.csv')

### 데이터 전처리

In [5]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            240 non-null    object 
 1   island             240 non-null    object 
 2   sex                232 non-null    object 
 3   bill_length_mm     238 non-null    float64
 4   bill_depth_mm      238 non-null    float64
 5   flipper_length_mm  238 non-null    float64
dtypes: float64(3), object(3)
memory usage: 11.4+ KB
None


<div class="alert alert-success">
    240개의 행과 6개의 컬럼으로 이루어져 있다. 6개의 변수를 독립 변수로 두고 분석을 진행한다. <br/> 
    sex에 8개, bill_length_mm, bill_depth_mm, flipper_length_mm에 각각 2개의 결측치가 있음을 확인할 수 있다. <br/>
</div>

- 결측치가 존재하는 데이터를 직접 확인해본다.

In [6]:
train = pd.concat([X_train, y_train], axis=1)    # y_train 데이터를 포함하여 결측치 확인을 해야 한다. axis=1 조건을 넣었기 때문에 두 데이터프레임이 좌우로 붙는다.
print(train.loc[(train.sex.isna()) | (train.bill_length_mm.isna()) | (train.bill_depth_mm.isna()) | (train.flipper_length_mm.isna()) | (train.body_mass_g.isna())])

    species     island  sex  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen  NaN            42.0           20.2              190.0   
6    Gentoo     Biscoe  NaN            44.5           14.3              216.0   
43   Gentoo     Biscoe  NaN             NaN            NaN                NaN   
66   Adelie  Torgersen  NaN            37.8           17.3              180.0   
88   Gentoo     Biscoe  NaN            47.3           13.8              216.0   
89   Adelie  Torgersen  NaN            37.8           17.1              186.0   
110  Gentoo     Biscoe  NaN            44.5           15.7              217.0   
229  Adelie  Torgersen  NaN             NaN            NaN                NaN   

     body_mass_g  
0         4250.0  
6         4100.0  
43           NaN  
66        3700.0  
88        4725.0  
89        3300.0  
110       4875.0  
229          NaN  


- 8개의 결측치를 제거하는 방식을 선택했다.

In [7]:
train = train.dropna()
train.reset_index(drop=True, inplace=True)    # 제거된 행의 인덱스가 비어 있는 상태로 데이터가 유지되므로, 인덱스를 초기화한다.

- 결측치 처리를 완료했으므로 붙였던 학습용 데이터를 다시 **독립 변수(`X`)** 와 **종속 변수(`y`)** 로 분리시킨다.

In [8]:
X_train = train[['species', 'island', 'sex', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y_train = train[['body_mass_g']]

In [9]:
print(X_train.describe())

       bill_length_mm  bill_depth_mm  flipper_length_mm
count      232.000000     232.000000         232.000000
mean        43.990948      17.226293         200.681034
std          5.509760       1.964677          14.064231
min         32.100000      13.200000         172.000000
25%         39.200000      15.700000         190.000000
50%         44.950000      17.350000         197.000000
75%         48.775000      18.725000         212.250000
max         58.000000      21.500000         231.000000


- 이상치가 없는 것으로 보이며, 각 수치형 변수의 범위가 상이하기 때문에 추후 스케일링이 필요한 것을 확인할 수 있다.

In [10]:
COL_DEL = []
COL_NUM = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']    # 수치형 변수
COL_CAT = ['species', 'island', 'sex']    # 범주형 변수
COL_Y = ['body_mass_g']

- 수치형 변수는 모델링을 진행하면서 스케일링을 진행할 예정이다.
    - 스케일링은 평균과 분산에 영향을 줄 수 있기 때문에 학습 데이터로만 진행한다.
- 범주형 변수에 대해 레이블 인코딩이 필요하다.
    - 레이블 인코딩은 현재 단계에서 모든 데이터를 포함하여 진행한다.

In [11]:
X = pd.concat([X_train, X_test])    # axis=1 조건을 넣지 않아 기본으로 설정된 조건에 따라 상하로 붙는다.

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X[COL_CAT])

X_train_res = ohe.transform(X_train[COL_CAT])    # 학습 데이터
X_test_res = ohe.transform(X_test[COL_CAT])    # 테스트 데이터

In [12]:
print(X_train_res)

  (0, 2)	1.0
  (0, 3)	1.0
  (0, 6)	1.0
  (1, 0)	1.0
  (1, 5)	1.0
  (1, 7)	1.0
  (2, 1)	1.0
  (2, 4)	1.0
  (2, 7)	1.0
  (3, 2)	1.0
  (3, 3)	1.0
  (3, 7)	1.0
  (4, 0)	1.0
  (4, 4)	1.0
  (4, 6)	1.0
  (5, 2)	1.0
  (5, 3)	1.0
  (5, 7)	1.0
  (6, 2)	1.0
  (6, 3)	1.0
  (6, 6)	1.0
  (7, 0)	1.0
  (7, 5)	1.0
  (7, 6)	1.0
  (8, 2)	1.0
  :	:
  (223, 6)	1.0
  (224, 2)	1.0
  (224, 3)	1.0
  (224, 7)	1.0
  (225, 0)	1.0
  (225, 4)	1.0
  (225, 7)	1.0
  (226, 1)	1.0
  (226, 4)	1.0
  (226, 7)	1.0
  (227, 1)	1.0
  (227, 4)	1.0
  (227, 6)	1.0
  (228, 2)	1.0
  (228, 3)	1.0
  (228, 7)	1.0
  (229, 0)	1.0
  (229, 5)	1.0
  (229, 6)	1.0
  (230, 1)	1.0
  (230, 4)	1.0
  (230, 6)	1.0
  (231, 2)	1.0
  (231, 3)	1.0
  (231, 6)	1.0


- `transform`된 데이터는 `numpy array`로 만들어진 데이터이며, 기존에 사용하던 데이터프레임이 아닌 특정 행렬 좌표에 1, 0 값이 존재한다는 방식으로 표현한 희소 행렬을 효율적으로 줄여서 표현한 데이터이다.
- 이것을 기존의 데이터에 적용하기 위해서는 데이터프레임으로 변환시키는 작업이 필요하다.

In [15]:
X_train_ohe = pd.DataFrame(X_train_res.todense(), columns=ohe.get_feature_names_out())    # todense 함수를 붙이고 get_feature_names_out 함수를 사용하여 변환된 범주형 변수를 만들 수 있다.
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns=ohe.get_feature_names_out())
print(X_train_ohe)

     species_Adelie  species_Chinstrap  species_Gentoo  island_Biscoe  \
0               0.0                0.0             1.0            1.0   
1               1.0                0.0             0.0            0.0   
2               0.0                1.0             0.0            0.0   
3               0.0                0.0             1.0            1.0   
4               1.0                0.0             0.0            0.0   
..              ...                ...             ...            ...   
227             0.0                1.0             0.0            0.0   
228             0.0                0.0             1.0            1.0   
229             1.0                0.0             0.0            0.0   
230             0.0                1.0             0.0            0.0   
231             0.0                0.0             1.0            1.0   

     island_Dream  island_Torgersen  sex_FEMALE  sex_MALE  
0             0.0               0.0         1.0       0.0  
1  

- `OneHotEncoding`을 통해 생성한 데이터와 수치형 변수로만 이루어진 두 데이터를 `concat` 함수를 통해 합쳐준다.

In [16]:
X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

### 데이터 모형 구축

#### 데이터 분할

In [17]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size=0.3)

#### 스케일링

- 수치형 변수에 대해 스케일링을 진행한다.

In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_tr[COL_NUM])

X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test_fin[COL_NUM] = scaler.transform(X_test_fin[COL_NUM])


<div class="alert alert-warning">
    fit 함수를 사용할 때는 X_tr 데이터만을 사용하도록 한다. 그래야만 평균과 분산에 영향을 미치는 데이터 누수가 발생하지 않는다.
</div>

### 모델 학습

In [19]:
from sklearn.linear_model import LinearRegression

modelLR = LinearRegression()
modelLR.fit(X_tr, y_tr)

y_val_pred = modelLR.predict(X_val)
print(y_val_pred)

[[3424.]
 [4280.]
 [4668.]
 [5316.]
 [3936.]
 [4076.]
 [5268.]
 [4112.]
 [4196.]
 [4048.]
 [3876.]
 [3316.]
 [4340.]
 [3372.]
 [4952.]
 [3344.]
 [3900.]
 [3392.]
 [4180.]
 [4632.]
 [3900.]
 [5368.]
 [3460.]
 [3760.]
 [4084.]
 [3556.]
 [5384.]
 [5392.]
 [3924.]
 [3304.]
 [4176.]
 [3468.]
 [3568.]
 [5240.]
 [3924.]
 [3496.]
 [5416.]
 [3096.]
 [4712.]
 [5516.]
 [3292.]
 [3476.]
 [4140.]
 [3400.]
 [5652.]
 [3320.]
 [4724.]
 [3300.]
 [3860.]
 [5376.]
 [3340.]
 [3312.]
 [5428.]
 [4996.]
 [3104.]
 [4724.]
 [4720.]
 [3296.]
 [4148.]
 [4032.]
 [5520.]
 [4268.]
 [4596.]
 [3696.]
 [4856.]
 [3472.]
 [4816.]
 [4812.]
 [5332.]
 [5408.]]


In [20]:
print(modelLR.intercept_)    # 절편 구하기

# 기울기 구하기
coef = pd.Series(data=modelLR.coef_[0], index=X_train_fin.columns)
print(coef.sort_values()) 

[2.46357982e+16]
species_Chinstrap   -2.373728e+16
species_Adelie      -2.373728e+16
species_Gentoo      -2.373728e+16
island_Dream        -8.985170e+14
island_Torgersen    -8.985170e+14
island_Biscoe       -8.985170e+14
sex_FEMALE          -1.853699e+02
sex_MALE             1.773699e+02
bill_depth_mm        3.719805e+02
bill_length_mm       7.234520e+02
flipper_length_mm    9.738607e+02
dtype: float64
